In [1]:
%reload_ext autoreload
%autoreload 2

from Load import *
import numpy as np
import os
import biosignalsnotebooks as bsnb
import sys
import matplotlib.pyplot as plt
module_path = os.path.abspath(os.path.join('C:\\Users\\Rodrigo\\Desktop\\PhD\\1st Study'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Process_Features import *

In [16]:
data, header = pyxdf.load_xdf('C:\\Users\\Rodrigo\\Desktop\\PhD\\LSL Client\\Data\\P001.xdf')

In [14]:
data1, header1 = pyxdf.load_xdf('C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\P0.xdf')

In [22]:
data

[{'info': defaultdict(list,
              {'name': ['Emteq.ExampleStream'],
               'type': ['Emteq.StreamType'],
               'channel_count': ['9'],
               'channel_format': ['float32'],
               'source_id': [None],
               'nominal_srate': ['50.00000000000000'],
               'version': ['1.100000000000000'],
               'created_at': ['98406.94558180000'],
               'uid': ['b060181c-684d-45e0-bd65-2f83d52b551b'],
               'session_id': ['default'],
               'hostname': ['rodrigo-yoga7'],
               'v4address': [None],
               'v4data_port': ['16573'],
               'v4service_port': ['16573'],
               'v6address': [None],
               'v6data_port': ['16573'],
               'v6service_port': ['16573'],
               'desc': [defaultdict(list,
                            {'channels': [defaultdict(list,
                                          {'channel': [defaultdict(list,
                                 

In [15]:
data1

[{'info': defaultdict(list,
              {'name': ['SAM'],
               'type': ['Rating'],
               'channel_count': ['2'],
               'channel_format': ['float32'],
               'source_id': [None],
               'nominal_srate': ['50.00000000000000'],
               'version': ['1.100000000000000'],
               'created_at': ['78669.55849230000'],
               'uid': ['280e36f7-3549-4a96-9c9b-24125ad493fb'],
               'session_id': ['default'],
               'hostname': ['rodrigo-yoga7'],
               'v4address': [None],
               'v4data_port': ['16572'],
               'v4service_port': ['16572'],
               'v6address': [None],
               'v6data_port': ['16572'],
               'v6service_port': ['16572'],
               'desc': [defaultdict(list,
                            {'channels': [defaultdict(list,
                                          {'channel': [defaultdict(list,
                                                        {'l

In [19]:
ratings={}
for stream in data1:
    if stream["info"]["name"][0] == "SAM":
        for i in range(0, int(stream["info"]["channel_count"][0])):
            ratings["CH" + str(i+1)] = stream["time_series"][:, i]

In [20]:
ratings

{'CH1': array([1., 1.], dtype=float32), 'CH2': array([2., 2.], dtype=float32)}

In [4]:
Signals,EEG_Signals,marker,timestamps = Load_Data(data)

TypeError: Load_Data() missing 5 required positional arguments: 'openSignals_stream_name', 'EEG_stream_name', 'markers_stream_name', 'ratings_stream_name', and 'sensors'

In [18]:
Emteq_Data={}
for stream in data:
    if stream["info"]["name"][0] == "Emteq.ExampleStream":
        time = stream["time_stamps"]
        for i in range(0, int(stream["info"]["channel_count"][0]) - 1):
            Emteq_Data["CH" + str(i + 1)] = stream["time_series"][:, i + 1]
print(Emteq_Data)

{'CH1': array([38., 41., 41., ..., 94., 90., 87.], dtype=float32), 'CH2': array([32., 33., 31., ..., 79., 77., 78.], dtype=float32), 'CH3': array([   0.,    0.,    0., ..., 1029., 1038., 1021.], dtype=float32), 'CH4': array([252., 240., 241., ..., 288., 295., 295.], dtype=float32), 'CH5': array([57., 55., 50., ..., 76., 79., 81.], dtype=float32), 'CH6': array([24., 28., 28., ..., 52., 59., 65.], dtype=float32), 'CH7': array([23840., 22512., 22512., ..., 23216., 22208., 22208.], dtype=float32), 'CH8': array([40368., 40448., 40448., ..., 40576., 40368., 40368.], dtype=float32)}


In [7]:
%matplotlib
plt.figure()
plt.plot(Emteq_Data["CH1"])
plt.plot(Emteq_Data["CH2"])
plt.plot(Emteq_Data["CH3"])
plt.plot(Emteq_Data["CH4"])
plt.plot(Emteq_Data["CH5"])
plt.plot(Emteq_Data["CH6"])
plt.plot(Emteq_Data["CH7"],label="CH7")
plt.plot(Emteq_Data["CH8"])
plt.legend()

Using matplotlib backend: QtAgg


In [38]:
marker,timestamps = Load_Psychopy(data)
CH1,CH2,CH3,CH4,CH5,CH6,time_Opensignals,fs = Load_Opensignals(data)
EEG_data,time_EEG,EEG_fs = Load_EEG(data)

In [39]:
onset,offset,videos = getMarkers(marker,timestamps)
len(onset),len(offset)

(26, 26)

In [40]:
onset_index,offset_index = getMarkersIndex(onset,offset,time_Opensignals)
onset_index_EEG,offset_index_EEG = getMarkersIndex(onset,offset,time_EEG)

In [41]:
timestamps-=time_Opensignals[0]
time_Opensignals -= time_Opensignals[0]
time_EEG -= time_EEG[0]

In [42]:
#Opensignals Data Plots
%matplotlib
plt.figure()
for timestamp,markers in zip(timestamps,marker):
    if (markers[1] == '1'):
        plt.axvline(x=timestamp,color='r')
    elif(markers[1] == '0'):
        plt.axvline(x=timestamp,color='g')
plt.plot(time_Opensignals,CH1,label="ECG")
plt.plot(time_Opensignals,CH2,label="EDA")
plt.plot(time_Opensignals,CH3,label="RESP")
plt.plot(time_Opensignals,CH4,label="TEMP")
plt.plot(time_Opensignals,CH5,label="fNIRS RED")
plt.plot(time_Opensignals,CH6,label="fNIRS IRED")
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.xlabel("Time (s)", fontsize=30)
plt.ylabel("Amplitude",fontsize=30)
# plt.legend()
# #EEG Data Plots
# plt.figure(2)
# for timestamp,markers in zip(timestamps,marker):
#     if (markers[1] == '1'):
#         plt.axvline(x=timestamp,color='r')
#     elif(markers[1] == '0'):
#         plt.axvline(x=timestamp,color='g')
# for keys in EEG_data.keys():
#     plt.plot(time_EEG,EEG_data[keys]-np.mean(EEG_data[keys]),'k',label=keys)

Using matplotlib backend: Qt5Agg


Text(0, 0.5, 'Amplitude')